# Digital Human Teleportation 项目 Colab Demo

## 1. 准备工作

首先，你应该通过github获取本项目代码：

In [ ]:
!git clone https://github.com/jzyustc/digital_human_teleportation.git

然后进入并设定当前目录为项目目录：

In [ ]:
import os
os.chdir("/content/digital_human_teleportation/")
results_path = "results/"
!pwd

安装所需要的环境：

In [ ]:
!pip install pymatting
!pip install --upgrade streamlit
!pip install pyngrok

将所需要的模型下载到对应目录：

In [ ]:
# model for MODNet
!mkdir MODNet/results/models/
!gdown --id 1YagzxHPOItCC0A-4TPQlOa7jdD-lW03z -O MODNet/results/models/modnet_photographic_portrait_matting.ckpt
# model for iDIH
!mkdir iDIH/results/models/
!gdown --id 1RhN9cZgdEEA-7FcaaPno8qE6wTApSgna -O iDIH/results/models/hrnet18s_idih256.pth

## 2. 运行默认 Demo

你可以直接运行得到和 Github 一样的效果图，输出将储存在 "results/output.png"

In [ ]:
# download 3d models
ply_path = "3d_models/"
!mkdir results/3d_models/
!gdown --id 1E3Z__qVZhvRcTGDxCEI5ptoJaddWnxPx -O results/3d_models/bg.ply
!gdown --id 1qDtdQFDR29YkgQD04haguRUhseJQ7jTV -O results/3d_models/bg_after.ply

# run
!python step1_matting.py
!python step2_ply2bg.py
!python step3_harmonization.py

# show result
import cv2
from google.colab.patches import cv2_imshow
output = cv2.imread("results/output.png")
cv2_imshow(output)

## 3. Colab 运行 Demo

#### 你可以选择：
1. 上传含有人像的图像文件：

In [ ]:
import shutil
from google.colab import files
from PIL import Image

# upload images (PNG or JPG)
image_names = list(files.upload().keys())
for image_name in image_names:
  shutil.move(image_name, os.path.join(results_path, image_name))

# only the first image is used as the foreground image
fg_image_name = os.path.join(results_path, image_names[0])
fg_image = Image.open(fg_image_name)

2. 使用默认的人像图像（results/human.jpg）

In [ ]:
from PIL import Image
fg_image_name = os.path.join(results_path, "human.jpg")
fg_image = Image.open(fg_image_name)

#### 下载 3D 模型

In [ ]:
ply_path = "3d_models/"
!mkdir results/3d_models/
!gdown --id 1E3Z__qVZhvRcTGDxCEI5ptoJaddWnxPx -O results/3d_models/bg.ply
!gdown --id 1qDtdQFDR29YkgQD04haguRUhseJQ7jTV -O results/3d_models/bg_after.ply

#### 设定参数

In [ ]:
# define path
ply_sub_path = "bg_after.ply"  # which 3d model to be used
output_name = "output.png"  # the saved output name

# define params
k = 580  # the size of pixels
image_range = [[-240, 240], [-320, 320]]  # camera plane range (pixels)
human_height = 1.0  # height of human image (m)
image_height = 0.3  # height of background projected into the camera plane (m)
center_pos = (160, 20)  # position of the human (pixels)
z = -6  # depth of the human (m)

# other params (not suggest to be changed)
f = -1  # depth of the camera plane
method = "quick"  # 3d model rendering method

#### 运行并输出

In [ ]:
from step1_matting import step1_matting
from step2_ply2bg import step2_ply2bg
from step3_harmonization import step3_harmonization

step1_matting(fg_image, results_path=results_path)
step2_ply2bg(os.path.join(ply_path, ply_sub_path), k=k, f=f, image_range=image_range, method=method, results_path=results_path)
output = step3_harmonization(output_name, human_height, image_height, center_pos, z, f=f, results_path=results_path)

from google.colab.patches import cv2_imshow
cv2_imshow(output)

## 4. Web 运行 Demo

后台运行 streamlit 交互式项目 Demo

In [ ]:
!streamlit run demo.py --server.port 80 &>/dev/null&

开通网页转换，允许公网访问

In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 80
public_url = ngrok.connect(port='80')
public_url

In [ ]:
!ps -aux | grep streamlit
!ps -aux | grep ngrok